In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from termcolor import colored
import requests
import io
import time
import os
import json
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
from functools import lru_cache

class StockAnalyzer:
    def __init__(self):
        self.base_dir = 'stock_analysis'
        self.stock_lists_dir = os.path.join(self.base_dir, 'stock_lists')
        self.results_dir = os.path.join(self.base_dir, 'daily_results')
        self.delisted_cache_file = os.path.join(self.stock_lists_dir, 'delisted_stocks.json')
        self.setup_directories()
        self.load_delisted_stocks()
        self.session = requests.Session()


    def setup_directories(self):
        """Create necessary directories if they don't exist"""
        os.makedirs(self.stock_lists_dir, exist_ok=True)
        os.makedirs(self.results_dir, exist_ok=True)
        
    def should_update_stock_list(self, exchange):
        """Check if stock list needs updating (weekly update)"""
        list_file = os.path.join(self.stock_lists_dir, f'{exchange}_stocks.json')
        if not os.path.exists(list_file):
            return True
        # Check if file is older than 7 days
        file_time = datetime.fromtimestamp(os.path.getmtime(list_file))
        return (datetime.now() - file_time).days >= 7
    
    def load_delisted_stocks(self):
        """Load previously identified delisted stocks"""
        try:
            with open(self.delisted_cache_file, 'r') as f:
                self.delisted_stocks = json.load(f)
        except FileNotFoundError:
            self.delisted_stocks = {'NSE': [], 'BSE': []}  # Changed from set to list
            self.save_delisted_stocks()

    def save_delisted_stocks(self):
        """Save delisted stocks to cache"""
        with open(self.delisted_cache_file, 'w') as f:
            json.dump({
                'NSE': self.delisted_stocks['NSE'],
                'BSE': self.delisted_stocks['BSE']
            }, f)
    
    def save_stock_list(self, symbols, exchange):
        """Save stock list to JSON file"""
        list_file = os.path.join(self.stock_lists_dir, f'{exchange}_stocks.json')
        data = {
            'symbols': symbols,
            'last_updated': datetime.now().strftime('%Y-%m-%d')
        }
        with open(list_file, 'w') as f:
            json.dump(data, f)

    def load_stock_list(self, exchange):
        """Load stock list from JSON file"""
        list_file = os.path.join(self.stock_lists_dir, f'{exchange}_stocks.json')
        with open(list_file, 'r') as f:
            data = json.load(f)
        return data['symbols']
    
    def download_nse_stocks(self):
        """Download NSE stock list if needed"""
        if self.should_update_stock_list('NSE'):
            try:
                url = "https://archives.nseindia.com/content/equities/EQUITY_L.csv"
                response = requests.get(url)
                df = pd.read_csv(io.StringIO(response.content.decode('utf-8')))
                symbols = df['SYMBOL'].tolist()
                self.save_stock_list(symbols, 'NSE')
                print("NSE stock list updated")
                return symbols
            except Exception as e:
                print(f"Error downloading NSE stocks: {e}")
                if os.path.exists(os.path.join(self.stock_lists_dir, 'NSE_stocks.json')):
                    return self.load_stock_list('NSE')
                return []
        return self.load_stock_list('NSE')

    def download_bse_stocks(self):
        """Download BSE stock list if needed"""
        if self.should_update_stock_list('BSE'):
            try:
                # Using the BSE list from static file or an appropriate source
                url = "https://www.bseindia.com/corporates/List_Scrips.aspx"
                response = requests.get(url)
                soup = BeautifulSoup(response.content, 'html.parser')
                table = soup.find('table', {'id': 'ContentPlaceHolder1_tblData'})
                symbols = []

                if table:
                    rows = table.find_all('tr')[1:]  # Skip header row
                    for row in rows:
                        cols = row.find_all('td')
                        if cols:
                            bse_code = cols[0].text.strip()
                            symbols.append(bse_code)

                if symbols:
                    self.save_stock_list(symbols, 'BSE')
                    print("BSE stock list updated")
                    return symbols
            except Exception as e:
                print(f"Error downloading BSE stocks: {e}")
                if os.path.exists(os.path.join(self.stock_lists_dir, 'BSE_stocks.json')):
                    return self.load_stock_list('BSE')
                return []
        return self.load_stock_list('BSE')

    @lru_cache(maxsize=1000)
    def calculate_rsi(self, prices_tuple, periods=14):
        try:
            prices = pd.Series(prices_tuple)
            
            # Check if we have enough data
            if len(prices) < periods + 1:  # Need at least periods + 1 data points
                print(f"Warning: Insufficient data for {periods}-period RSI calculation. Need at least {periods + 1} points, got {len(prices)}")
                
                # If we have at least 5 data points, calculate with adjusted period
                if len(prices) >= 6:
                    adjusted_periods = min(5, len(prices) - 1)
                    print(f"Calculating RSI with adjusted period of {adjusted_periods}")
                    return self._calculate_rsi_core(prices, adjusted_periods)
                return None
                
            return self._calculate_rsi_core(prices, periods)
            
        except Exception as e:
            print(f"Error calculating RSI: {e}")
            return None

    def _calculate_rsi_core(self, prices, periods):
        """
        Core RSI calculation with improved gain/loss handling
        
        Args:
            prices: Pandas Series of price values
            periods: RSI period
            
        Returns:
            float: RSI value
        """
        # Calculate price changes
        delta = prices.diff()
        
        # Split gains and losses
        gains = delta.copy()
        losses = delta.copy()
        
        gains[gains < 0] = 0
        losses[losses > 0] = 0
        losses = abs(losses)  # Convert losses to positive values
        
        # Calculate initial averages
        first_avg_gain = gains[:periods+1].mean()
        first_avg_loss = losses[:periods+1].mean()
        
        # Initialize lists to store values
        avg_gains = [first_avg_gain]
        avg_losses = [first_avg_loss]
        
        # Calculate subsequent values
        for i in range(periods+1, len(gains)):
            avg_gain = (avg_gains[-1] * (periods-1) + gains[i]) / periods
            avg_loss = (avg_losses[-1] * (periods-1) + losses[i]) / periods
            avg_gains.append(avg_gain)
            avg_losses.append(avg_loss)
        
        # Calculate final RSI
        if avg_losses[-1] == 0:
            rsi = 100.0
        else:
            rs = avg_gains[-1] / avg_losses[-1]
            rsi = 100 - (100 / (1 + rs))
        
        return float(rsi)

    def calculate_ema(self, data, length, source='Close', offset=0, smoothing_length=9):
        """
        Calculate Exponential Moving Average with specified parameters
        
        Args:
            data (pd.DataFrame): Historical price data
            length (int): Length of EMA period
            source (str): Source column for calculation
            offset (int): Offset for calculation
            smoothing_length (int): Length of initial SMA smoothing
            
        Returns:
            pd.Series: EMA values
        """
        if isinstance(data, pd.Series):
            series_data = data
        else:
            series_data = data[source]
            
        # First calculate SMA as the smoothing line
        sma = series_data.rolling(window=smoothing_length).mean()
        
        # Calculate the multiplier
        multiplier = 2 / (length + 1)
        
        # Initialize EMA with SMA
        ema = pd.Series(index=series_data.index, dtype=float)
        ema.iloc[:length-1] = np.nan
        ema.iloc[length-1] = sma.iloc[length-1]
        
        # Calculate EMA
        for i in range(length, len(series_data)):
            ema.iloc[i] = (series_data.iloc[i] - ema.iloc[i-1]) * multiplier + ema.iloc[i-1]
        
        # Apply offset if specified
        if offset != 0:
            ema = ema.shift(offset)
            
        return ema

    def calculate_tema(self, data, length, source='Close'):
        """
        Calculate the Triple Exponential Moving Average (TEMA) for the given data.
        
        Formula: TEMA = (3 * EMA1) - (3 * EMA2) + EMA3
        Where:
            EMA1 = EMA(price, length)
            EMA2 = EMA(EMA1, length)
            EMA3 = EMA(EMA2, length)
        """
        # Get price series
        if isinstance(data, pd.Series):
            prices = data
        else:
            prices = data[source]
        
        # Calculate multiplier
        multiplier = 2 / (length + 1)
        
        # Calculate EMA1
        ema1 = prices.copy()
        for i in range(1, len(prices)):
            ema1.iloc[i] = (prices.iloc[i] * multiplier) + (ema1.iloc[i-1] * (1 - multiplier))
        
        # Calculate EMA2
        ema2 = ema1.copy()
        for i in range(1, len(ema1)):
            ema2.iloc[i] = (ema1.iloc[i] * multiplier) + (ema2.iloc[i-1] * (1 - multiplier))
        
        # Calculate EMA3
        ema3 = ema2.copy()
        for i in range(1, len(ema2)):
            ema3.iloc[i] = (ema2.iloc[i] * multiplier) + (ema3.iloc[i-1] * (1 - multiplier))
        
        # Calculate TEMA
        tema = (3 * ema1) - (3 * ema2) + ema3
        
        # Replace first 'length' periods with NaN as they won't have enough data
        tema.iloc[:length-1] = np.nan
        
        return tema

    def get_hourly_ema21(self, symbol, exchange='NSE'):
        """Calculate EMA-21 based on 1-hour candles"""
        try:
            # Handle different ticker symbols for NSE and BSE
            if exchange == 'NSE':
                ticker_symbol = f"{symbol}.NS"
            else:  # BSE
                # For BSE stocks, we need to handle the format differently
                ticker_symbol = f"{symbol}.BO"
                
            ticker = yf.Ticker(ticker_symbol)
            
            # Get hourly data for the last 30 days (to have enough data for EMA calculation)
            end_date = datetime.now()
            start_date = end_date - timedelta(days=30)
            
            hourly_data = ticker.history(
                start=start_date,
                end=end_date,
                interval="1h"
            )
            
            if hourly_data.empty or len(hourly_data) < 21:
                print(f"Insufficient hourly data for {symbol} ({exchange})")
                return None
            
            # Calculate EMA-21 on hourly data
            hourly_ema21 = self.calculate_ema(hourly_data, length=21, source='Close')
            
            # Return the latest EMA-21 value
            return hourly_ema21.iloc[-1]
            
        except Exception as e:
            print(f"Error calculating hourly EMA-21 for {symbol} ({exchange}): {e}")
            return None
        
    def get_stock_data(self, symbol, exchange='NSE', days=200):
        """Fetch detailed stock data with additional information"""
        if symbol in self.delisted_stocks[exchange]:
            return None, None, None, None

        try:
            time.sleep(1)  # Add a delay of 1 second between requests
            ticker_symbol = f"{symbol}.NS" if exchange == 'NSE' else f"{symbol}.BO"
            ticker = yf.Ticker(ticker_symbol)
            
            # Get stock info
            info = ticker.info
            
            end_date = datetime.now()
            start_date = end_date - timedelta(days=days)

            hist_data = ticker.history(
                start=start_date,
                end=end_date,
                interval="1d"
            )

            if hist_data.empty:
                return None, None, None, None

            # Calculate EMAs
            if len(hist_data) >= 9:
                hist_data['EMA_9'] = self.calculate_ema(hist_data, length=9, source='Close')
            else:
                hist_data['EMA_9'] = float('nan')
                
            if len(hist_data) >= 21:
                hist_data['EMA_21'] = self.calculate_ema(hist_data, length=21, source='Close')
                # Add hourly EMA-21
                hourly_ema21 = self.get_hourly_ema21(symbol, exchange)
                hist_data['Hourly_EMA_21'] = hourly_ema21 if hourly_ema21 is not None else float('nan')
            else:
                hist_data['EMA_21'] = float('nan')
                hist_data['Hourly_EMA_21'] = float('nan')
                
            if len(hist_data) >= 50:
                try:
                    hist_data['EMA_50'] = self.calculate_ema(hist_data, length=50, source='Close')
                    hist_data['TEMA_50'] = self.calculate_tema(hist_data, length=50, source='Close')
                except Exception as e:
                    print(f"Error calculating TEMA for {symbol}: {e}")
                    hist_data['EMA_50'] = float('nan')
                    hist_data['TEMA_50'] = float('nan')
            else:
                hist_data['EMA_50'] = float('nan')
                hist_data['TEMA_50'] = float('nan')
            
            latest_price = hist_data['Close'].iloc[-1]
            prices_tuple = tuple(hist_data['Close'].values)
            
            # Improved RSI calculation with data length check
            if len(prices_tuple) < 3:  # Minimum required data points
                latest_rsi = None
            else:
                latest_rsi = self.calculate_rsi(prices_tuple)
                if latest_rsi is not None:
                    latest_rsi = round(latest_rsi, 2)  # Round to 2 decimal places

            return latest_price, hist_data, latest_rsi, info

        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
            return None, None, None, None

    def process_stock_batch(self, batch_data):
        """Process a batch of stocks with enhanced information"""
        symbols, exchange = batch_data
        results = []
        
        with ThreadPoolExecutor(max_workers=8) as executor:
            futures = {executor.submit(self.get_stock_data, symbol, exchange): symbol 
                      for symbol in symbols}
            
            for future in futures:
                symbol = futures[future]
                try:
                    latest_price, hist_data, latest_rsi, info = future.result()
                    if all(v is not None for v in [latest_price, hist_data, latest_rsi, info]):
                        # Extract stock information
                        stock_info = {
                            'Symbol': symbol,
                            'Exchange': exchange,
                            'Company_Name': info.get('longName', 'N/A'),
                            'Latest_Price': latest_price,
                            'EMA_9': hist_data['EMA_9'].iloc[-1] if 'EMA_9' in hist_data else 'N/A',
                            'EMA_21': hist_data['EMA_21'].iloc[-1] if 'EMA_21' in hist_data else 'N/A',
                            'Hourly_EMA_21': hist_data['Hourly_EMA_21'].iloc[-1] if 'Hourly_EMA_21' in hist_data else 'N/A',
                            'EMA_50': hist_data['EMA_50'].iloc[-1] if 'EMA_50' in hist_data else 'N/A',
                            'TEMA_50': hist_data['TEMA_50'].iloc[-1] if 'TEMA_50' in hist_data else 'N/A',
                            'RSI': latest_rsi if latest_rsi is not None else 'Insufficient Data',
                            'Volume': hist_data['Volume'].iloc[-1] if 'Volume' in hist_data else 0,
                            'Market_Cap': info.get('marketCap', 'N/A'),
                            'PE_Ratio': info.get('trailingPE', 'N/A'),
                            'EPS': info.get('trailingEps', 'N/A'),
                            'Dividend_Yield': info.get('dividendYield', 'N/A'),
                            'Book_Value': info.get('bookValue', 'N/A'),
                            'Sector': info.get('sector', 'N/A'),
                            'Industry': info.get('industry', 'N/A'),
                            '52W_High': info.get('fiftyTwoWeekHigh', 'N/A'),
                            '52W_Low': info.get('fiftyTwoWeekLow', 'N/A'),
                            '50d_MA': info.get('fiftyDayAverage', 'N/A'),
                            '200d_MA': info.get('twoHundredDayAverage', 'N/A'),
                            'Beta': info.get('beta', 'N/A'),
                            'Previous_Close': info.get('previousClose', 'N/A'),
                            'Open': info.get('open', 'N/A'),
                            'Day_High': info.get('dayHigh', 'N/A'),
                            'Day_Low': info.get('dayLow', 'N/A'),
                            'Date': datetime.now().strftime('%Y-%m-%d')
                        }
                        
                        # Calculate additional metrics
                        if hist_data is not None:
                            stock_info.update({
                                'Volatility_30d': hist_data['Close'].pct_change().std() * np.sqrt(252),
                                'Returns_30d': (hist_data['Close'].iloc[-1] / hist_data['Close'].iloc[0] - 1) * 100,
                                'Average_Volume_30d': hist_data['Volume'].mean(),
                                'LTP > EMA_50': latest_price > stock_info['EMA_50'],
                                'TEMA_50 > EMA_50': stock_info['TEMA_50'] > stock_info['EMA_50'],
                            })
                        
                        results.append(stock_info)
                        
                except Exception as e:
                    print(f"Error processing {symbol}: {e}")

        return results


    def run_daily_analysis(self):
        """Run daily analysis with Excel output"""
        current_date = datetime.now().strftime('%Y%m%d')
        
        print("Loading stock lists...")
        
        bse_symbols = self.download_bse_stocks()
        nse_symbols = self.download_nse_stocks()
        

        print(f"\nAnalyzing {len(nse_symbols)} NSE and {len(bse_symbols)} BSE stocks")

        for exchange, symbols in [('NSE', nse_symbols), ('BSE', bse_symbols)]:
            batch_data=(symbols, exchange)
            results = self.process_stock_batch(batch_data)
            
            if results:
                # Create Excel writer with xlsxwriter engine
                excel_file = os.path.join(
                    self.results_dir,
                    f'{exchange.lower()}_analysis_{current_date}.xlsx'
                )
                
                # Convert results list to DataFrame
                results_df = pd.DataFrame(results)
                
                with pd.ExcelWriter(excel_file, engine='xlsxwriter') as writer:
                    # Write all stocks to first sheet
                    results_df.to_excel(writer, sheet_name='All Stocks', index=False)
                    
                    # Write high RSI stocks to second sheet
                    high_rsi = results_df[results_df['RSI'] >= 40].sort_values('RSI', ascending=False)
                    high_rsi.to_excel(writer, sheet_name='High RSI Stocks', index=False)
                    
                    # Get workbook and worksheet objects for formatting
                    workbook = writer.book
                    
                    # Add formats
                    header_format = workbook.add_format({
                        'bold': True,
                        'text_wrap': True,
                        'valign': 'top',
                        'bg_color': '#D9E1F2',
                        'border': 1
                    })
                    
                    # Format each worksheet
                    for worksheet in writer.sheets.values():
                        # Set column widths
                        worksheet.set_column('A:Z', 15)
                        # Apply header format to first row
                        for col_num, value in enumerate(results_df.columns.values):
                            worksheet.write(0, col_num, value, header_format)

                print(f"\n{exchange} analysis complete. Results saved to {excel_file}")
                
                # Print high RSI stocks
                print(f"\n{exchange} Stocks with RSI >= 40:")
                for _, row in high_rsi.iterrows():
                    print(colored(
                        f"{row['Symbol']} ({row['Company_Name']}): RSI = {row['RSI']:.2f}, "
                        f"Price = ₹{row['Latest_Price']:.2f}",
                        'green', attrs=['bold']
                    ))
    
        print(f"\nDaily analysis complete. Results saved in {self.results_dir}")

if __name__ == "__main__":
    analyzer = StockAnalyzer()
    analyzer.run_daily_analysis()

Loading stock lists...

Analyzing 2061 NSE and 3026 BSE stocks


CAMLIN-RE.NS: Period '1mo' is invalid, must be one of ['1d', '5d']


Calculating RSI with adjusted period of 5


DHAN-RE.NS: Period '1mo' is invalid, must be one of ['1d', '5d']
$HEROMOTOCO.NS: possibly delisted; no price data found  (1d 2024-07-06 16:35:44.673050 -> 2025-01-22 16:35:44.673050)


Calculating RSI with adjusted period of 5


...

Insufficient hourly data for 513554 (BSE)
Insufficient hourly data for 513519 (BSE)


$513605.BO: possibly delisted; no price data found  (1d 2024-07-06 16:48:32.819304 -> 2025-01-22 16:48:32.819304)


Insufficient hourly data for 513599 (BSE)


$513683.BO: possibly delisted; no timezone found
$513691.BO: possibly delisted; no price data found  (1d 2024-07-06 16:48:33.244374 -> 2025-01-22 16:48:33.244374)


Insufficient hourly data for 513729 (BSE)
Insufficient hourly data for 514167 (BSE)
Insufficient hourly data for 514175 (BSE)
Insufficient hourly data for 514211 (BSE)
Insufficient hourly data for 514234 (BSE)
Insufficient hourly data for 514286 (BSE)
Insufficient hourly data for 514300 (BSE)
Insufficient hourly data for 514274 (BSE)
Insufficient hourly data for 514348 (BSE)
Insufficient hourly data for 514418 (BSE)Insufficient hourly data for 514354 (BSE)

Insufficient hourly data for 515030 (BSE)
Insufficient hourly data for 515037 (BSE)
Insufficient hourly data for 515093 (BSE)
Insufficient hourly data for 515145 (BSE)
Insufficient hourly data for 516016 (BSE)
Insufficient hourly data for 516092 (BSE)
Insufficient hourly data for 516022 (BSE)
Insufficient hourly data for 517015 (BSE)
Insufficient hourly data for 517041 (BSE)
Insufficient hourly data for 517146 (BSE)
Insufficient hourly data for 517174 (BSE)
Insufficient hourly data for 517271 (BSE)
Insufficient hourly data for 51738

In [2]:
import yfinance as yf

symbol = 'RELIANCE.BO'  # Replace with the desired BSE stock symbol
interval = '1h'  # Set the interval to hourly
stock = yf.Ticker(symbol)
intraday_data = stock.history(period='1d', interval=interval)
print(intraday_data)


                                  Open         High          Low        Close  \
Datetime                                                                        
2025-01-21 09:15:00+05:30  1312.500000  1312.849976  1285.650024  1287.300049   
2025-01-21 10:15:00+05:30  1286.000000  1286.849976  1278.050049  1285.800049   
2025-01-21 11:15:00+05:30  1285.900024  1286.400024  1282.550049  1283.849976   

                           Volume  Dividends  Stock Splits  
Datetime                                                    
2025-01-21 09:15:00+05:30   63222        0.0           0.0  
2025-01-21 10:15:00+05:30  114396        0.0           0.0  
2025-01-21 11:15:00+05:30    5214        0.0           0.0  
